In [1]:
from haversine import haversine_vector, Unit
import os
import numpy as np
from cmdstanpy import cmdstan_path, CmdStanModel

import arviz as az
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

import cmdstanpy
cmdstanpy.install_cmdstan()

Installing CmdStan version: 2.31.0
Install directory: /Users/andrea/.cmdstan
CmdStan version 2.31.0 already installed


True

In [2]:
import pandas as pd

source = "../house12latlong.csv"
df = pd.read_csv(source, index_col=0)
df['Latitude'] = df['Latitude'].str.replace(',', '.').astype(float)
df['Longitude'] = df['Longitude'].str.replace(',', '.').astype(float)
df

,price,Overall.Qual,Garage.Area,Total.Bsmt.SF,X1st.Flr.SF,Full.Bath,AgeofHouse,Mas.Vnr.Area,TotRms.AbvGrd,Fireplaces,Wood.Deck.SF,Open.Porch.SF,Longitude,Latitude
1,215000,6,528.0,1080.0,1656,1,50,112.0,7,2,210,62,-93.619754,42.054035
2,149900,6,452.0,1178.0,1293,2,52,28.0,5,0,280,0,-93.618462,42.053406
3,157500,6,313.0,1008.0,1024,1,50,244.0,5,2,0,0,-93.618182,42.053327
4,124500,6,502.0,882.0,882,1,53,0.0,5,0,0,0,-93.618895,42.053036
5,169000,5,963.0,1617.0,1797,1,49,302.0,7,1,0,0,-93.619262,42.053108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,170000,5,418.0,1389.0,1389,1,31,0.0,6,1,240,38,-93.600190,41.990921
2916,285000,8,482.0,1679.0,1803,2,15,0.0,6,2,129,64,-93.600006,41.989848
2917,188000,7,650.0,996.0,996,2,12,94.0,9,1,190,48,-93.599996,41.989265
2918,231000,6,616.0,1152.0,1152,2,16,0.0,7,1,162,312,-93.600147,41.989185


In [4]:
indexes = df.index.copy().to_numpy()
np.random.shuffle(indexes)
df = df.loc[indexes[:500], :]

coords = df.loc[:, ["Latitude", "Longitude"]]
dists = haversine_vector(coords, coords, Unit.KILOMETERS, comb=True)

In [7]:
N = df.shape[0]
phi=0.8
delta = 1e-5
Sigma = np.ndarray((N,N))

for i in range(0, N):
    for j in range (0,N):
        Sigma[i, j] = np.exp((-1)*phi*dists[i,j]);
    Sigma[i, i]=Sigma[i,i]+delta;

Tau = np.linalg.inv(Sigma) 